# SCSProx Demo
Compute proximal operators of CVXPY problems very quickly, using cached matrix factorizations, warm-starting, and 
re-stuffing which avoids the overhead of CVXPY.

- for the demo, first initialize a simple CVXPY problem

In [1]:
from scsprox import Prox
import numpy as np
import cvxpy as cvx

m, n = 400, 200

x = cvx.Variable(n)

np.random.seed(1)
A = np.random.randn(m,n)
b = np.random.randn(m)

prob = cvx.Problem(cvx.Minimize(cvx.norm(A*x-b)))
prob.solve()

x_true = np.array(x.value).flatten()
#x_true

## form the `Prox` object
- create the prox object by passing in the CVXPY problem, along with a dict, `prox_vars`, of the proximal variables
- during initialization, the `Prox` object forms a CySCS `Workspace`, which computes and stores the SCS factorization (which only needs to be computed once)
- the `Prox` object accepts arbitrary CVXPY problems and any dict of related CVXPY variables to form the prox

In [2]:
prox_vars = {'x': x}
prox = Prox(prob, prox_vars)

----------------------------------------------------------------------------
	SCS v1.2.6 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2016
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 80203
eps = 1.00e-03, alpha = 1.50, max_iters = 2500, normalize = 1, scale = 1.00
Variables n = 202, constraints m = 603
Cones:	soc vars: 603, soc blks: 2
Setup time: 3.45e-02s


/Users/ajfriend/miniconda3/envs/env3/lib/python3.5/site-packages/cyscs/_util.py:165: UserWarning: Converting A.indptr and A.indices to arrays with dtype = numpy.int64
  warn("Converting A.indptr and A.indices to arrays with dtype = numpy.int64")


## evaluate the prox
- evaluate the prox, noting that SCS **doesn't** initialize, because the factorization has been cached
- note that this first call to `prox.do()` takes 40 iterations
- the prox operator is evaluted on `x0`, a dictionary of variable names and values (matching the names and variable sizes in `prox_vars`)
- within SCS, the `A` matrix doesn't change between prox calls, but the `b` and `c` vectors do change
    - `Prox` automatically restuffs the values in `b` and `c` based on the data in `x0`
    - restuffing is just an **indexing** operation, and **avoids the overhead** of CVXPY entirely

In [3]:
x0 = {'x': np.zeros(n)}
rho = 1.0
x1 = prox.do(x0, rho)

SCS using variable warm-starting
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 1.63e+00  8.85e+00  9.89e-01 -2.84e+01  6.39e+01  0.00e+00  2.27e-03 
    40| 2.52e-07  1.68e-06  4.28e-07  1.43e+01  1.43e+01  3.05e-15  2.57e-02 
----------------------------------------------------------------------------
Status: Solved
Timing: Solve time: 2.57e-02s
	Lin-sys: nnz in L factor: 100908, avg solve time: 5.56e-04s
	Cones: avg projection time: 1.78e-06s
----------------------------------------------------------------------------
Error metrics:
dist(s, K) = 0.0000e+00, dist(y, K*) = 0.0000e+00, s'y/|s||y| = -4.4263e-16
|Ax + s - b|_2 / (1 + |b|_2) = 2.5241e-07
|A'y + c|_2 / (1 + |c|_2) = 1.6757e-06
|c'x + b'y| / (1 + |c'x| + |b'y|) = 4.2803e-07
--------------------------------------------------------------

## automatic warm-starting
- if we call `prox.do()` again, we can take advantage of warm-starting
- the `Prox` object caches the last SCS solution, and uses it to warm-start the next SCS solve
- with the same `x0` as the previous call, SCS does **0** iterations, because the warm-start solution from the previous iteration is already within the solver tolerance

In [4]:
x0 = {'x': np.zeros(n)}
rho = 1.0
x1 = prox.do(x0, rho)

SCS using variable warm-starting
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 1.92e-07  1.17e-06  8.76e-09  1.43e+01  1.43e+01  0.00e+00  2.98e-03 
----------------------------------------------------------------------------
Status: Solved
Timing: Solve time: 3.00e-03s
	Lin-sys: nnz in L factor: 100908, avg solve time: 1.18e-03s
	Cones: avg projection time: 4.77e-06s
----------------------------------------------------------------------------
Error metrics:
dist(s, K) = 4.4409e-15, dist(y, K*) = 2.2204e-16, s'y/|s||y| = -6.0345e-16
|Ax + s - b|_2 / (1 + |b|_2) = 1.9154e-07
|A'y + c|_2 / (1 + |c|_2) = 1.1677e-06
|c'x + b'y| / (1 + |c'x| + |b'y|) = 8.7562e-09
----------------------------------------------------------------------------
c'x = 14.3283, -b'y = 14.3283


## warm-starting 2
- of course, we usually won't try to compute the prox on exactly the same value, but instead, a slight perturbation of that value
- warm-starting still helps in this case, and still works automatically
- we call the prox on `x1`, the output of the first prox computation
- SCS is warm-started from the previous solution, which will tend to reduce the number of iterations needed

In [5]:
rho = 1.0
x2 = prox.do(x1, rho)

SCS using variable warm-starting
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 7.79e-02  1.80e+00  1.27e-01  1.38e+01  1.06e+01  0.00e+00  3.04e-03 
    20| 9.58e-06  2.06e-04  1.53e-06  1.39e+01  1.39e+01  4.76e-15  9.94e-03 
----------------------------------------------------------------------------
Status: Solved
Timing: Solve time: 9.96e-03s
	Lin-sys: nnz in L factor: 100908, avg solve time: 3.51e-04s
	Cones: avg projection time: 1.53e-06s
----------------------------------------------------------------------------
Error metrics:
dist(s, K) = 1.7764e-15, dist(y, K*) = 4.4409e-16, s'y/|s||y| = -4.4795e-16
|Ax + s - b|_2 / (1 + |b|_2) = 9.5789e-06
|A'y + c|_2 / (1 + |c|_2) = 2.0606e-04
|c'x + b'y| / (1 + |c'x| + |b'y|) = 1.5295e-06
--------------------------------------------------------------

## proximal iteration
- as an example application, we can solve the original CVXPY problem through proximal iteration

In [6]:
for i in range(20):
    x0 = prox.do(x0, rho, verbose=False)

- note that, after several iterations, proximal iteration converges, and the SCS solver finishes in **0** iterations

In [7]:
x0 = prox.do(x0, rho, verbose=True)

SCS using variable warm-starting
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 3.77e-06  8.07e-05  2.22e-06  1.39e+01  1.39e+01  0.00e+00  2.53e-03 
----------------------------------------------------------------------------
Status: Solved
Timing: Solve time: 2.55e-03s
	Lin-sys: nnz in L factor: 100908, avg solve time: 7.10e-02s
	Cones: avg projection time: 2.41e-04s
----------------------------------------------------------------------------
Error metrics:
dist(s, K) = 2.2204e-16, dist(y, K*) = 2.7756e-17, s'y/|s||y| = 6.6904e-16
|Ax + s - b|_2 / (1 + |b|_2) = 3.7744e-06
|A'y + c|_2 / (1 + |c|_2) = 8.0667e-05
|c'x + b'y| / (1 + |c'x| + |b'y|) = 2.2226e-06
----------------------------------------------------------------------------
c'x = 13.8774, -b'y = 13.8774


- we can do the same thing within an ADMM iteration:
    - set the tolerance to, say, 1e-5
    - set the maximum number of SCS iterations (per prox) to, say, 10 or 100
    - this results in ADMM with approximate proximal evaluations (only a few SCS iterations), but which eventually converges (via warm-starting) to the desired tolerance of 1e-5